In [9]:
corpus_chapters_lemmas_dictionary.filter_extremes(no_below=5, no_above=0.5)
corpus_chapters_lemmas_bow =  [corpus_chapters_lemmas_dictionary.doc2bow(chapter) 
                      for chapter in corpus_chapters_lemmas
                              ]

# Make an index to word dictionary.
temp = corpus_chapters_lemmas_dictionary[0]  # This is only to "load" the dictionary.
id2word = corpus_chapters_lemmas_dictionary.id2token

for num_topic in num_topics:
    lda_model_lemmas = LdaModel(
        corpus=corpus_chapters_lemmas_bow,
        id2word=id2word,
        chunksize=chunksize,
        alpha='auto',
        eta='auto',
        iterations=iterations,
        num_topics=num_topic,
        passes=passes,
        eval_every=eval_every
    )
    coherence_model_lda = CoherenceModel(model=lda_model_lemmas, texts=corpus_chapters_lemmas, 
                                         dictionary=corpus_chapters_lemmas_dictionary,
                                         coherence='c_v')
    
    print("topics: " + str(num_topic) ,"coherence: ", coherence_model_lda.get_coherence(), "\tperplexity: ", 
          lda_model_lemmas.log_perplexity(corpus_chapters_lemmas_bow))

NameError: name 'corpus_chapters_lemmas_dictionary' is not defined

In [11]:
import os
from lxml import etree
import gensim
#read the corpus
src_file = os.path.join('src/DLUG_Ann_clean.vrt')
tree = etree.parse(src_file)
root = tree.getroot()

In [12]:
## liber year div/chapter,head,p
def check_type(div):
    typeof = 'text'  #p div etc
    if div.tag == "head":
        if "class" in div.attrib.keys():
            if div.attrib["class"] == "annus":
                typeof='annus'
            elif div.attrib["class"] == "chapter":
                typeof='chapter'
            else:
                typeof="head"
        else:
                typeof="head"                
    return typeof

def process_vrt(raw):
    lines = []
    for line in [line for line in raw.split("\n") if line != ""]:
        tok,pos,lem = None, None, None
        cols = line.split('\t')
        if len(cols) == 3:
            tok, pos,lem = cols
        else:
            tok = cols[0]
        lines.append((tok,pos,lem))
    return lines
    #return raw
        
text_dict = {}
#text_dict = {book : {chapter : [{year, typeof, txt}]} }
liber = 0
for el in root: #liber
    year = 0
    chapter = None
    classes = ['head', 'annus', 'text']
    typeof = 'text'
    raw_text = ''
    
    #liber
    if "class" in el.attrib.keys():
        if el.attrib["class"] == 'liber':
            liber = int(el.attrib["n"])
            text_dict[liber] = dict()
            #print("liber",liber)
    for ell in el.getchildren():
        if "class" in el.attrib.keys():
            if ell.attrib["class"] == "year":
                year = ell.attrib["n"]
                #print("year ", year)
                for elll in ell.getchildren():
                    if elll.tag == "head":
                        typeof = check_type(elll)                
                        raw_text = elll.text
                        
                        if chapter is not None:
                            chapter += 1
                        else:
                            chapter = 0
                        
                        text_dict[liber][chapter] = list()
                        text_dict[liber][chapter].append([year, typeof, process_vrt(raw_text)])
                            
                    elif elll.tag == "div":
                        if elll.attrib["class"] == "chapter":
                            if chapter is not None:
                                chapter += 1
                            else:
                                chapter = 0
                            #print("chapter ", chapter)
                            text_dict[liber][chapter] = list()
                            for ellll in elll.getchildren():
                                typeof = check_type(ellll)
                                raw_text = ellll.text
                                #print("chapter ", chapter)                           
                                text_dict[liber][chapter].append([year, typeof, process_vrt(raw_text)])
                                
                                
                    
            #chapters under liber 0-1
            elif ell.attrib["class"] == "chapter":
                if chapter is not None:
                    chapter += 1
                else:
                    chapter = 0
                text_dict[liber][chapter] = list()
                for elll in ell.getchildren():
                    typeof = check_type(elll)
                    raw_text = elll.text
                    text_dict[liber][chapter].append([year, typeof, process_vrt(raw_text)])
                    
                    #print("raw_text ", raw_text)
                    #print("typeof ", typeof)
#print(text_dict[4][5]) #chapter (head, text*) in a single year
#print(type(text_dict))

In [13]:
docs_by_year = [] 
docs_by_liber = []
docs_by_chapter = [] # list of chapters (liber, chapter, year, tokens)
#for liber in [2]:
tmp = []
for liber in text_dict.keys():
    for chapter in text_dict[liber].keys():
        chapter_tokens = []
        for txt_arr in text_dict[liber][chapter]:
            #txt_arr_toks = []
            if txt_arr[1] == "text": #exclude head etc.   
                #print(txt_arr)
                for tok in txt_arr[2]: #iterates over tokens                    
                    chapter_tokens.append((liber, chapter, txt_arr[0], tok[0], tok[1], tok[2])) #liber,chap, year,token
            else:
                tmp.append(txt_arr)            
                
        if len(chapter_tokens) > 0:
            docs_by_chapter.append(chapter_tokens)

In [14]:
#liber year div/chapter,head,p
def check_type(div):
    typeof = 'text'  #p div etc
    if div.tag == "head":
        if "class" in div.attrib.keys():
            if div.attrib["class"] == "annus":
                typeof='annus'
            elif div.attrib["class"] == "chapter":
                typeof='chapter'
            else:
                typeof="head"
        else:
                typeof="head"                
    return typeof

def process_vrt(raw):
    lines = []
    for line in [line for line in raw.split("\n") if line != ""]:
        tok,pos,lem = None, None, None
        cols = line.split('\t')
        if len(cols) == 3:
            tok, pos,lem = cols
        else:
            tok = cols[0]
        lines.append((tok,pos,lem))
    return lines
    #return raw
        
text_dict = {}
#text_dict = {book : {chapter : [{year, typeof, txt}]} }
liber = 0
for el in root: #liber
    year = 0
    chapter = None
    classes = ['head', 'annus', 'text']
    typeof = 'text'
    raw_text = ''
    
    #liber
    if "class" in el.attrib.keys():
        if el.attrib["class"] == 'liber':
            liber = int(el.attrib["n"])
            text_dict[liber] = dict()
            #print("liber",liber)
    for ell in el.getchildren():
        if "class" in el.attrib.keys():
            if ell.attrib["class"] == "year":
                year = ell.attrib["n"]
                #print("year ", year)
                for elll in ell.getchildren():
                    if elll.tag == "head":
                        typeof = check_type(elll)                
                        raw_text = elll.text
                        
                        if chapter is not None:
                            chapter += 1
                        else:
                            chapter = 0
                        
                        text_dict[liber][chapter] = list()
                        text_dict[liber][chapter].append([year, typeof, process_vrt(raw_text)])
                            
                    elif elll.tag == "div":
                        if elll.attrib["class"] == "chapter":
                            if chapter is not None:
                                chapter += 1
                            else:
                                chapter = 0
                            #print("chapter ", chapter)
                            text_dict[liber][chapter] = list()
                            for ellll in elll.getchildren():
                                typeof = check_type(ellll)
                                raw_text = ellll.text
                                #print("chapter ", chapter)                           
                                text_dict[liber][chapter].append([year, typeof, process_vrt(raw_text)])
                                
                                
                    
            #chapters under liber 0-1
            elif ell.attrib["class"] == "chapter":
                if chapter is not None:
                    chapter += 1
                else:
                    chapter = 0
                text_dict[liber][chapter] = list()
                for elll in ell.getchildren():
                    typeof = check_type(elll)
                    raw_text = elll.text
                    text_dict[liber][chapter].append([year, typeof, process_vrt(raw_text)])
                    
                    #print("raw_text ", raw_text)
                    #print("typeof ", typeof)
#print(text_dict[4][5]) #chapter (head, text*) in a single year
#print(type(text_dict))

docs_by_year = [] 
docs_by_liber = []

In [15]:
docs_by_chapter = [] # list of chapters (liber, chapter, year, tokens)
all_by_chapter= [] #all chaps
txt_chapt_numbers = []
txt_chapt_num_current = []
text_to_all_chapters = {} #{chapter : {liber:chapter}} #only texts
text_dict_joined = {}

i = 0
for liber in text_dict.keys():
    if liber not in text_dict_joined.keys():
        text_dict_joined[liber] = {}
    for chapter in text_dict[liber].keys():
        if chapter not in text_dict_joined[liber].keys():
            text_dict_joined[liber][chapter] = {"year":None}
            
        
        chapter_tokens_txt = []
        chapter_tokens = []
        
        for txt_arr in text_dict[liber][chapter]:
            text_dict_joined[liber][chapter]["year"] = txt_arr[0]
            if txt_arr[1] not in text_dict_joined[liber][chapter].keys():
                text_dict_joined[liber][chapter][txt_arr[1]] = []
            text_dict_joined[liber][chapter][txt_arr[1]].append(' '.join([tok[0] for tok in txt_arr[2]]))
            for tok in txt_arr[2]: #iterates over tokens
                chapter_tokens.append((liber, chapter, txt_arr[0], txt_arr[1], tok[0], tok[1], tok[2])) #liber,chap, year,token
            
            if txt_arr[1] == "text": #exclude head etc.
                if str(liber) + "_" + str(chapter) not in txt_chapt_num_current:
                    txt_chapt_num_current.append(str(liber) + "_" + str(chapter))
                    text_to_all_chapters[i] = {liber:chapter}
                    i+= 1
                for tok in txt_arr[2]: #iterates over tokens
                    chapter_tokens_txt.append((liber, chapter, txt_arr[0], tok[0], tok[1], tok[2])) #liber,chap, year,token
        if len(chapter_tokens_txt) > 0:
            docs_by_chapter.append(chapter_tokens_txt)
        all_by_chapter.append(chapter_tokens)

In [16]:
corpus_chapters = []
corpus_chapters_lemmas = [] #no uknown
corpus_chapters_lemmas_all = []
corpus_chapters_lemmas_year = []
year = 0
for chap in docs_by_chapter:
    doc = []
    doc_lemmas = []
    doc_lemmas_all = []
    
    for tok in chap:
        if tok[4] not in ["PUN", "SENT"]:
            doc.append(tok[3].lower())            
            if tok[5] is not None and tok[5] != 'UNKNOWN':
                doc_lemmas.append(tok[5].lower())
                year = tok[2]
            elif tok[5] is not None:
                if tok[5] == 'UNKNOWN':
                    doc_lemmas_all.append(tok[3].lower())
                else:
                    doc_lemmas_all.append(tok[5].lower())
    corpus_chapters_lemmas_year.append(year)
    corpus_chapters.append(doc)
    corpus_chapters_lemmas.append(doc_lemmas)
    corpus_chapters_lemmas_all.append(doc_lemmas_all)

In [18]:
corpus_chapters_lemmas_reduced = []
stoplist_lemma = ["-", "major", "nemo", "aliter", "nisi", "verum", "quis", "per", "vir", "teneo", "saltim", "velut", "ita", 
                  "quis", "do", "etenim", "vix", "bene", "vel","facio|fio", "quoniam", "se", "a", "de", "cum","tuus", 
                  "omnis", "ex", "sive", "post", "ego", "tu", "non", "quia", "eo", "sum", "qui", "ut", "ac", "is", 
                  "ille","ad","in","et","sed", "que", "deus", "tam", "hic", "homo", "tempus", "aut", "atque", "habeo",
                  "dum", "unus", "ipse", "nec", "quoque", "adeo", "aput", "alius", "nullus", "enim", "genus", "autem", 
                  "quidem", "res", "mille", "si", "plus", "possum", "tamen", "ubi", "sed", "multus", "idem", "tunc",
                  "propter", "fero", "tam", "ne", "noster", "nos", "dominus", "ago", "contra", "res", "inter", "etiam", 
                  "sub", "duo", "tres", "septem", "quattuor", "quinque", "sex", "septem", "octo", "novem", "decem", 
                  "viginti", "ceterus", "annus", "circa", "homo", "ab", "sus", "insuper", "ab", "suus", "pro", "alter", "seu", "item", "quidam",
                 "magis", "facio", "dico", "appello", "itaque", "jam", "credo", "dies", "magis", "siquidem"]
for chapter in corpus_chapters_lemmas:
    corpus_chapters_lemmas_reduced.append([tok for tok in chapter if tok not in stoplist_lemma])

In [19]:
from gensim import corpora
corpus_chapters_lemmas_reduced_dictionary = corpora.Dictionary(corpus_chapters_lemmas_reduced)
corpus_chapters_lemmas_reduced_bow =  [corpus_chapters_lemmas_reduced_dictionary.doc2bow(chapter) 
                      for chapter in corpus_chapters_lemmas_reduced]
#corpus_chapters_lemmas_reduced_dictionary.filter_extremes(no_below=5, no_above=0.5)

In [20]:
#fasttext
from gensim.models import FastText
fasttext_dlugosz = FastText(vector_size=100, window=3, min_count=1)
fasttext_dlugosz.build_vocab(corpus_iterable=corpus_chapters_lemmas_reduced)
fasttext_dlugosz.train(corpus_iterable=corpus_chapters_lemmas_reduced, total_examples=len(corpus_chapters_lemmas_reduced), epochs=10)

(3912634, 4146770)

In [22]:
fasttext_dlugosz.wv.similar_by_word('gaudium')

[('studium', 0.9607945084571838),
 ('tripudium', 0.9368204474449158),
 ('leodium', 0.9139331579208374),
 ('repudium', 0.8765716552734375),
 ('suspirium', 0.8738833069801331),
 ('brevium', 0.8712905049324036),
 ('gremium', 0.867394208908081),
 ('exordium', 0.8673692941665649),
 ('cilicium', 0.8643236756324768),
 ('natalicium', 0.8638047575950623)]

In [23]:
from gensim.models import Word2Vec
word2vec_dlugosz = Word2Vec(vector_size=100, window=5, min_count=5)
word2vec_dlugosz.build_vocab(corpus_iterable=corpus_chapters_lemmas_reduced)
word2vec_dlugosz.train(corpus_iterable=corpus_chapters_lemmas_reduced, total_examples=len(corpus_chapters_lemmas_reduced), epochs=10)

(3832148, 4146770)

In [24]:
word2vec_dlugosz.wv.similar_by_word('gaudium')

[('hilaritas', 0.902108371257782),
 ('exulto', 0.8064785003662109),
 ('pompa', 0.8009859323501587),
 ('celebritas', 0.7931421995162964),
 ('honos', 0.7905219197273254),
 ('laus', 0.7862314581871033),
 ('redemptor', 0.7818967700004578),
 ('gratitudo', 0.7778508067131042),
 ('tripudium', 0.770086944103241),
 ('adaugeo', 0.765107274055481)]

In [25]:
word2vec_dlugosz.wv.similar_by_word('civitas')

[('arcs', 0.7771523594856262),
 ('arx', 0.7371326684951782),
 ('claves|clavis', 0.7300114035606384),
 ('applico', 0.727290153503418),
 ('impono', 0.7163296937942505),
 ('urbs', 0.7152277231216431),
 ('porta', 0.7023245096206665),
 ('villa|villus', 0.689978837966919),
 ('redigo', 0.6855376958847046),
 ('principalis', 0.6799649000167847)]

In [37]:
corpus_chapters[2596]

['die',
 'saturni',
 'quarta',
 'mensis',
 'aprilis',
 'alias',
 'sabbato',
 'ante',
 'conductum',
 'pasche',
 'ludovicus',
 'de',
 'erlichausen',
 'prussie',
 'magister',
 'vicesimus',
 'nonus',
 'stabilitis',
 'et',
 'compositis',
 'pulchro',
 'ordine',
 'commendariis',
 'et',
 'officiis',
 'ordinis',
 'et',
 'in',
 'ipsas',
 'commendarias',
 'et',
 'officia',
 'fratribus',
 'ex',
 'ordine',
 'qui',
 'sedato',
 'bello',
 'superstites',
 'fuerant',
 'suffectis',
 'pomerania',
 'et',
 'culmensi',
 'ac',
 'michaloviensi',
 'terris',
 'in',
 'corpus',
 'regni',
 'polonie',
 'redeuntibus',
 'novas',
 'commendarias',
 'et',
 'officia',
 'in',
 'terra',
 'prussie',
 'novamque',
 'sedem',
 'pro',
 'magistro',
 'aput',
 'kinsberg',
 'erigi',
 'et',
 'creari',
 'oportebat',
 'tactus',
 'infirmitate',
 'que',
 'diebus',
 'quindecim',
 'illum',
 'maceravit',
 'omnibus',
 'sacramentis',
 'potitus',
 'catholice',
 'et',
 'religiose',
 'aput',
 'kinsberg',
 'moritur',
 'et',
 'in',
 'ecclesia',
 's